In [37]:
# Imports
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Load
cars = pd.read_csv('/content/train.csv')

In [38]:
# Inspect
cars.info()
cars.head()
cars.describe(include='all')
cars.isna().sum().sort_values(ascending=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5847 non-null   int64  
 1   Name               5847 non-null   object 
 2   Location           5847 non-null   object 
 3   Year               5847 non-null   int64  
 4   Kilometers_Driven  5847 non-null   int64  
 5   Fuel_Type          5847 non-null   object 
 6   Transmission       5847 non-null   object 
 7   Owner_Type         5847 non-null   object 
 8   Mileage            5845 non-null   object 
 9   Engine             5811 non-null   object 
 10  Power              5811 non-null   object 
 11  Seats              5809 non-null   float64
 12  New_Price          815 non-null    object 
 13  Price              5847 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 639.6+ KB


,0
New_Price,5032
Seats,38
Power,36
Engine,36
Mileage,2
Unnamed: 0,0
Fuel_Type,0
Kilometers_Driven,0
Year,0
Location,0


In [39]:
# Copy
df = cars.copy()


In [40]:
# Helper to pick mean vs median based on skewness
def impute_num(series):
    if series.skew(skipna=True) > 1 or series.skew(skipna=True) < -1:
        return series.fillna(series.median())
    return series.fillna(series.mean())

num_cols = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price', 'Price']
cat_cols = ['Fuel_Type','Transmission','Location','Owner_Type']

In [57]:
# If any assumed columns differ, adjust lists above.

# Impute numerical
for c in num_cols:
    if c in df.columns:
        df[c] = impute_num(df[c])


In [43]:
# Helper function to clean and convert columns to numeric
def clean_and_convert_numeric(df, column, unit_to_remove=None):
    if column in df.columns and df[column].dtype == 'object':
        # Remove units and non-numeric characters
        df[column] = df[column].astype(str).str.replace(r'[a-zA-Z%/-]', '', regex=True)
        df[column] = df[column].astype(str).str.replace(',', '', regex=False)
        if unit_to_remove:
            df[column] = df[column].astype(str).str.replace(unit_to_remove, '', regex=False)
        # Convert to numeric, coercing errors to NaN
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

# Clean 'Mileage' column
df = clean_and_convert_numeric(df, 'Mileage', 'kmpl')

# Clean 'Engine' column
df = clean_and_convert_numeric(df, 'Engine', 'CC')

# Clean 'Power' column
df = clean_and_convert_numeric(df, 'Power', 'bhp')

# Clean 'New_Price' column (assuming it contains 'Rs.' and potentially commas)
df = clean_and_convert_numeric(df, 'New_Price', 'Rs.')

# Display the info and head of the cleaned columns to verify
print('Info after cleaning:')
df[['Mileage', 'Engine', 'Power', 'New_Price']].info()
print('\nHead of cleaned columns:')
display(df[['Mileage', 'Engine', 'Power', 'New_Price']].head())

Info after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Mileage    5845 non-null   float64
 1   Engine     5811 non-null   float64
 2   Power      5811 non-null   float64
 3   New_Price  815 non-null    float64
dtypes: float64(4)
memory usage: 182.8 KB

Head of cleaned columns:


,Mileage,Engine,Power,New_Price
0,19.67,1582.0,126.20,NaN
1,13.00,1199.0,88.70,8.61
2,20.77,1248.0,88.76,NaN
3,15.20,1968.0,140.80,NaN
4,23.08,1461.0,63.10,NaN


In [44]:
# Re-run the imputation for numerical columns after cleaning

# Impute numerical
for c in num_cols:
    if c in df.columns:
        df[c] = impute_num(df[c])

print('\nNaN values after imputation:')
display(df[num_cols].isna().sum().sort_values(ascending=False))


NaN values after imputation:


,0
Year,0
Kilometers_Driven,0
Mileage,0
Engine,0
Power,0
Seats,0
New_Price,0
Price,0


In [45]:
# Impute categorical with mode or ‘Unknown’ if multimodal/ties
for c in cat_cols:
    if c in df.columns:
        mode = df[c].mode(dropna=True)
        df[c] = df[c].fillna(mode.iloc[0] if not mode.empty else 'Unknown')



In [46]:
# Drop columns with extreme missingness
na_frac = df.isna().mean()
to_drop = na_frac[na_frac > 0.6].index.tolist()
df = df.drop(columns=to_drop)


Justification:

Numerical features were filled with either the mean or median depending on skewness to preserve central tendency, while categorical features were imputed with the mode to maintain category integrity. Columns with extreme missingness (>60%) were dropped to avoid introducing bias or unreliable imputations.


In [47]:
import re

def extract_number(x):
    if pd.isna(x): return np.nan
    # capture first numeric (with decimal) in string
    m = re.search(r'(\d+(?:\.\d+)?)', str(x))
    return float(m.group(1)) if m else np.nan

for col, unit in [('Mileage','kmpl'), ('Engine','CC'), ('Power','bhp'), ('New_price','lakh')]:
    if col in df.columns:
        df[col] = df[col].apply(extract_number)




In [48]:
# Re-impute after extraction to handle any new NaNs
for c in ['Mileage','Engine','Power','New_price']:
    if c in df.columns:
        df[c] = df[c].fillna(df[c].median())


In [49]:
ohe_cols = [c for c in ['Fuel_Type','Transmission'] if c in df.columns]
df_encoded = pd.get_dummies(df, columns=ohe_cols, drop_first=True)  # drop_first to avoid collinearity
df_encoded.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,11.48,12.50,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,False,True,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,11.48,6.00,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,11.48,17.74,False,False,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,11.48,3.50,False,False,True


In [50]:
from datetime import datetime
current_year = datetime.now().year
if 'Year' in df_encoded.columns:
    df_encoded['Car_Age'] = current_year - df_encoded['Year']


In [51]:
if set(['Price','Power']).issubset(df_encoded.columns):
    df_encoded['Price_per_bhp'] = df_encoded['Price'] / (df_encoded['Power'].replace(0, np.nan))
if set(['Odometer','Mileage']).issubset(df_encoded.columns):
    df_encoded['Odo_per_kmpl'] = df_encoded['Odometer'] / (df_encoded['Mileage'].replace(0, np.nan))


In [52]:
# Select
selected = df_encoded[['Location','Fuel_Type_Petrol']].dropna(how='all', axis=1) if 'Fuel_Type_Petrol' in df_encoded.columns else df_encoded[['Location']]

# Filter (e.g., cars newer than 2015 with automatic transmission)
mask = pd.Series(True, index=df_encoded.index)
if 'Year' in df_encoded.columns:
    mask &= (df_encoded['Year'] >= 2015)
auto_cols = [c for c in df_encoded.columns if c.startswith('Transmission_')]
if auto_cols:
    # assume one column is Transmission_Automatic after OHE
    auto_col = [c for c in auto_cols if 'Automatic' in c]
    if auto_col:
        mask &= (df_encoded[auto_col[0]] == 1)
filtered = df_encoded[mask]

In [53]:
# Rename
renamed = filtered.rename(columns={'Kilometers_Driven':'Km_Driven', 'New_Price':'NewPrice_Lakh'})

In [54]:
# Mutate (create/transform)
if 'Price' in renamed.columns and 'Car_Age' in renamed.columns:
    renamed['Price_per_Age'] = renamed['Price'] / renamed['Car_Age'].replace(0, np.nan)

In [55]:
# Arrange (sort)
arranged = renamed.sort_values(by=['Price','Car_Age'], ascending=[False, True])

In [56]:
# Summarize with groupby (e.g., by Location and Fuel type)
print("Columns in 'arranged' before summarization:", arranged.columns)
group_cols = [c for c in ['Location'] if c in arranged.columns]
summary = (arranged
           .groupby(group_cols)
           .agg(Avg_Price=('Price','mean'),
                Median_Odo=('Km_Driven','median'),
                Count=('Price','size'))
           .reset_index())
summary.head()

Columns in 'arranged' before summarization: Index(['Unnamed: 0', 'Name', 'Location', 'Year', 'Km_Driven', 'Owner_Type',
       'Mileage', 'Engine', 'Power', 'Seats', 'NewPrice_Lakh', 'Price',
       'Fuel_Type_Electric', 'Fuel_Type_Petrol', 'Transmission_Manual',
       'Car_Age', 'Price_per_bhp', 'Price_per_Age'],
      dtype='object')


,Location,Avg_Price,Median_Odo,Count
0,Ahmedabad,11.720732,42075.0,82
1,Bangalore,19.298990,38000.0,99
2,Chennai,12.008472,39000.0,144
3,Coimbatore,17.283696,36931.0,441
4,Delhi,12.552179,40000.0,179


In [59]:
df_encoded.to_csv('/content/train.csv', index=False)
